<a href="https://colab.research.google.com/github/SummerProjectTEAM-10/sample/blob/SRUTHI/One_Hot_Encoding%2BIQR%2BHyperParameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **IMPORT LIBRARIES AND DATASET**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats


from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

df=pd.read_csv('/content/pima-indians-diabetes.csv')
df.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


### **DATA PREPROCESSING**

In [2]:
dataset=df
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)
# The missing values ​​will be filled with the median values ​​of each variable.
def median_target(var):   
    temp = df[df[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp
# The values to be given for incomplete observations are given the median value of people who are not sick and the median values of people who are sick.
columns = df.columns
columns = columns.drop("Outcome")
for i in columns:
    median_target(i)
    df.loc[(df['Outcome'] == 0 ) & (df[i].isnull()), i] = median_target(i)[i][0]
    df.loc[(df['Outcome'] == 1 ) & (df[i].isnull()), i] = median_target(i)[i][1]



### **OUTLIER REJECTION USING IQR**

In [3]:

Q1=df.quantile(0.25)
Q3=df.quantile(0.75)
IQR=Q3-Q1

print("---Q1--- \n",Q1)
print("\n---Q3--- \n",Q3)
print("\n---IQR---\n",IQR)

---Q1--- 
 Pregnancies                   1.00000
Glucose                      99.75000
BloodPressure                64.00000
SkinThickness                25.00000
Insulin                     102.50000
BMI                          27.50000
DiabetesPedigreeFunction      0.24375
Age                          24.00000
Outcome                       0.00000
Name: 0.25, dtype: float64

---Q3--- 
 Pregnancies                   6.00000
Glucose                     140.25000
BloodPressure                80.00000
SkinThickness                32.00000
Insulin                     169.50000
BMI                          36.60000
DiabetesPedigreeFunction      0.62625
Age                          41.00000
Outcome                       1.00000
Name: 0.75, dtype: float64

---IQR---
 Pregnancies                  5.0000
Glucose                     40.5000
BloodPressure               16.0000
SkinThickness                7.0000
Insulin                     67.0000
BMI                          9.1000
DiabetesPed

In [4]:

#outlier remove
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.shape,df_out.shape

((768, 9), (590, 9))

### **FEATURE ENGINEERING**

In [5]:
# According to BMI, some ranges are determined and categorical variables are assigned.
NewBMI = pd.Series(["Underweight", "Healthy", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"], dtype = "category")
df["NewBMI"] = NewBMI
df.loc[df["BMI"] < 18.5, "NewBMI"] = NewBMI[0]
df.loc[(df["BMI"] > 18.5) & (df["BMI"] <= 24.9), "NewBMI"] = NewBMI[1]
df.loc[(df["BMI"] > 24.9) & (df["BMI"] <= 29.9), "NewBMI"] = NewBMI[2]
df.loc[(df["BMI"] > 29.9) & (df["BMI"] <= 34.9), "NewBMI"] = NewBMI[3]
df.loc[(df["BMI"] > 34.9) & (df["BMI"] <= 39.9), "NewBMI"] = NewBMI[4]
df.loc[df["BMI"] > 39.9 ,"NewBMI"] = NewBMI[5]

# A categorical variable creation process is performed according to the insulin value.
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"
df = df.assign(NewInsulinScore=df.apply(set_insulin, axis=1))

# Some intervals were determined according to the glucose variable and these were assigned categorical variables.
NewGlucose = pd.Series(["Low", "Normal", "Prediabetes", "High"], dtype = "category")
df["NewGlucose"] = NewGlucose
df.loc[df["Glucose"] <= 70, "NewGlucose"] = NewGlucose[0]
df.loc[(df["Glucose"] > 70) & (df["Glucose"] <= 99), "NewGlucose"] = NewGlucose[1]
df.loc[(df["Glucose"] > 99) & (df["Glucose"] <= 126), "NewGlucose"] = NewGlucose[2]
df.loc[df["Glucose"] > 126 ,"NewGlucose"] = NewGlucose[3]

df.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI,NewInsulinScore,NewGlucose
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1,Obesity 1,Abnormal,High
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0,Overweight,Normal,Normal
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1,Healthy,Abnormal,High


### **ONE HOT ENCODING**

In [6]:
from sklearn.preprocessing import OneHotEncoder
df1 = pd.get_dummies(df, columns =["NewBMI","NewInsulinScore", "NewGlucose"], drop_first = True)
categorical_df = df1[['NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Prediabetes']]
categorical_df.head(3)

,NewBMI_Obesity 1,NewBMI_Obesity 2,NewBMI_Obesity 3,NewBMI_Overweight,NewBMI_Underweight,NewInsulinScore_Normal,NewGlucose_Low,NewGlucose_Normal,NewGlucose_Prediabetes
0,1,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,0,1,0
2,0,0,0,0,0,0,0,0,0


In [7]:
df1.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI_Obesity 1,NewBMI_Obesity 2,NewBMI_Obesity 3,NewBMI_Overweight,NewBMI_Underweight,NewInsulinScore_Normal,NewGlucose_Low,NewGlucose_Normal,NewGlucose_Prediabetes
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1,1,0,0,0,0,0,0,0,0
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0,0,0,0,1,0,1,0,1,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1,0,0,0,0,0,0,0,0,0


### **DATA SPLITTING**

In [8]:
Features_Columns = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Prediabetes']
#Features_Columns = ['Pregnancies','Glucose','Insulin','BMI','DiabetesPedigreeFunction','Age','BMI_Lable','Insulin_Lable','Glucose_Lable']
X = df1[Features_Columns]  # INDEPENDENT VARIABLES
y = df1["Outcome"]         # DEPENDENT VARIABLE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)



### **LOGISTIC REGRESSION**

In [9]:
Logmodel = LogisticRegression()
Logmodel.fit(X_train, y_train)
y_prediction = Logmodel.predict(X_test)



accuracy1_LR = Logmodel.score(X_train, y_train)
print("Accuracy of train data = ", accuracy1_LR * 100, "%")
accuracy2_LR = Logmodel.score(X_test, y_test)
print("Accuracy of test data = ", accuracy2_LR * 100, "%")

Accuracy of train data =  83.79888268156425 %
Accuracy of test data =  85.28138528138528 %


In [10]:
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
param_grid_LR = {
    'solver' : ['newton-cg', 'lbfgs', 'liblinear','sag','saga'],
    'penalty' : ['l1', 'l2', 'elasticnet'],
    'C' : [100, 10, 1.0, 0.1, 0.01]
}
grid_LR = GridSearchCV(LogisticRegression(),param_grid_LR, scoring='accuracy', n_jobs=-1, cv=5).fit(X_train,y_train)
print('Best Score: %s' % grid_LR.best_score_)
print('Best Hyperparameters: %s' % grid_LR.best_params_)



Best Score: 0.841883004499827
Best Hyperparameters: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}


In [11]:
Tuned_LR_model = LogisticRegression(**grid_LR.best_params_).fit(X_train, y_train)
Tuned_LR_y_prediction = Tuned_LR_model.predict(X_test)


HP_accuracy1_LR = Tuned_LR_model.score(X_train, y_train)
print("Accuracy of train data = ", HP_accuracy1_LR * 100, "%")
HP_accuracy2_LR = Tuned_LR_model.score(X_test, y_test)
print("Accuracy of test data = ", HP_accuracy2_LR * 100, "%")


Accuracy of train data =  85.84729981378027 %
Accuracy of test data =  86.14718614718615 %


### **SVM**

In [12]:

SvcModel = SVC()
SvcModel.fit(X_train, y_train)
y_prediction_SVM = SvcModel.predict(X_test)

accuracy1_SVM = SvcModel.score(X_train, y_train)
print("Accuracy of train data = ", accuracy1_SVM * 100, "%")
accuracy2_SVM = SvcModel.score(X_test, y_test)
print("Accuracy of test data = ", accuracy2_SVM * 100, "%")

#from sklearn.metrics import classification_report, confusion_matrix
#print(classification_report(y_test, y_prediction_SVM))

Accuracy of train data =  85.6610800744879 %
Accuracy of test data =  81.38528138528139 %


In [13]:
param_grid_SVM = {'C': [0.01,0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','sigmoid']}
grid_SVM = GridSearchCV(SVC(), param_grid_SVM, refit = True,return_train_score=True, n_jobs=-1,cv=5).fit(X_train, y_train)

# print best parameter after tuning
print('Best Score: %s' % grid_SVM.best_score_)
print("BEST PARAMETER:",grid_SVM.best_params_) 
# print how our model looks after hyper-parameter tuning
#print(grid.best_estimator_)


Best Score: 0.8585496711664936
BEST PARAMETER: {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}


In [14]:
Tuned_SVM_model = SVC(**grid_SVM.best_params_).fit(X_train, y_train)
Tuned_SVM_y_prediction= Tuned_SVM_model.predict(X_test)

HP_accuracy1_SVM = Tuned_SVM_model.score(X_train, y_train)
print("Accuracy of train data = ", HP_accuracy1_SVM * 100, "%")
HP_accuracy2_SVM = Tuned_SVM_model.score(X_test, y_test)
print("Accuracy of test data = ", HP_accuracy2_SVM * 100, "%")
#print ("Accuracy : ", accuracy_score(y_test, Tuned_SVM_y_prediction)*100,"%")

#from sklearn.metrics import classification_report, confusion_matrix
#print(classification_report(y_test, HP_y_prediction_SVC))

Accuracy of train data =  87.33705772811918 %
Accuracy of test data =  82.25108225108225 %


### **DECISION TREE**

In [15]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train , y_train)
y_pred_decision_tree = decision_tree.predict(X_test)


accuracy1_DT = decision_tree.score(X_train, y_train)
print("Accuracy of train data = ", accuracy1_DT * 100, "%")
accuracy2_DT = decision_tree.score(X_test, y_test)
print("Accuracy of test data = ", accuracy2_DT * 100, "%")

Accuracy of train data =  100.0 %
Accuracy of test data =  83.98268398268398 %


In [16]:
param_grid_DT = {'criterion': ["gini", "entropy"], 
              'splitter': ["best", "random"], 
              'max_depth': [1, 2, 3], 
              'min_samples_split': [2, 3, 4]}
grid_DT = GridSearchCV(DecisionTreeClassifier(), param_grid_DT, cv=5, n_jobs=-1,scoring = 'accuracy' ).fit(X_train,y_train)


print ("Best score is:",grid_DT.best_score_) 
print ("BEST PARAMETERS:",grid_DT.best_params_)


#best_model = grid_DecisionTree.best_estimator_
#y_pred = best_model.predict(X_test)
#The accuracy of the test set:
#print ("The test set accuracy is:",accuracy_score(y_test, y_pred))

Best score is: 0.8679473866389754
BEST PARAMETERS: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2, 'splitter': 'best'}


In [17]:
Tuned_DT_model = DecisionTreeClassifier(**grid_DT.best_params_).fit(X_train,y_train)
Tuned_DT_y_prediction= Tuned_DT_model.predict(X_test)

HP_accuracy1_DT = Tuned_DT_model.score(X_train, y_train)
print("Accuracy of train data = ", HP_accuracy1_DT * 100, "%")
HP_accuracy2_DT = Tuned_DT_model.score(X_test, y_test)
print("Accuracy of test data = ", HP_accuracy2_DT * 100, "%")

#print ("Accuracy : ", accuracy_score(y_test, Tuned_DT_y_prediction)*100,"%")

Accuracy of train data =  88.8268156424581 %
Accuracy of test data =  86.14718614718615 %


### **NAIVE BAYES**

In [18]:
naive_bayes = GaussianNB()
naive_bayes.fit(X_train , y_train)
y_pred_naive_bayes = naive_bayes.predict(X_test)

accuracy1_NB = naive_bayes.score(X_train, y_train)
print("Accuracy of train data = ", accuracy1_NB * 100, "%")
accuracy2_NB = naive_bayes.score(X_test, y_test)
print("Accuracy of test data = ", accuracy2_NB * 100, "%")

Accuracy of train data =  57.16945996275605 %
Accuracy of test data =  62.77056277056276 %


In [19]:
np.logspace(0,-9, num=10)

array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07,
       1.e-08, 1.e-09])

In [20]:
from sklearn.preprocessing import PowerTransformer
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
grid_NB = GridSearchCV(estimator=GaussianNB(), param_grid=params_NB, cv=5, scoring='accuracy')
Data_transformed = PowerTransformer().fit_transform(X_test)
grid_NB.fit(Data_transformed, y_test)

print ("Best score is:",grid_NB.best_score_) 
print ("BEST PARAMETERS:",grid_NB.best_params_)

Best score is: 0.8139685476410732
BEST PARAMETERS: {'var_smoothing': 0.657933224657568}


In [21]:
predict_test = grid_NB.predict(Data_transformed)

# Accuracy Score on test dataset
accuracy_test = accuracy_score(y_test,predict_test)
print('accuracy_score on test dataset : ', accuracy_test)

accuracy_score on test dataset :  0.8181818181818182


In [33]:
Tuned_NB_model = GaussianNB(**grid_NB.best_params_).fit(X_train,y_train)
Tuned_NB_y_prediction= Tuned_NB_model.predict(Data_transformed)

HP_accuracy1_NB = Tuned_NB_model.score(X_train, y_train)
print("Accuracy of train data = ", HP_accuracy1_NB * 100, "%")
HP_accuracy2_NB = Tuned_NB_model.score(Data_transformed, y_test)
print("Accuracy of test data = ", HP_accuracy2_NB * 100, "%")

#print ("Accuracy : ", accuracy_score(y_test, Tuned_NB_y_prediction)*100,"%")

Accuracy of train data =  67.0391061452514 %
Accuracy of test data =  63.20346320346321 %


### **RANDOM FOREST**

In [23]:
Random_Forest = RandomForestClassifier()
Random_Forest.fit(X_train , y_train)
y_pred_Random_Forest = Random_Forest.predict(X_test)


accuracy1_RF = Random_Forest.score(X_train, y_train)
print("Accuracy of train data = ", accuracy1_RF * 100, "%")
accuracy2_RF = Random_Forest.score(X_test, y_test)
print("Accuracy of test data = ", accuracy2_RF * 100, "%")

Accuracy of train data =  100.0 %
Accuracy of test data =  88.74458874458875 %


In [24]:
param_grid_RF = {'bootstrap': [True],
    'max_depth': [80, 90, 100,110],
    'criterion' : ('gini','entropy'),
    'max_features': [2, 3],
    #'min_samples_leaf': [3, 4, 5],
    #'min_samples_split': [8, 10, 12],
    'n_estimators': [ 200, 300, 1000]}
grid_RF = GridSearchCV(RandomForestClassifier(), param_grid_RF, cv=5, n_jobs=-1,scoring = 'accuracy' ).fit(X_train,y_train)


print ("Best score is:",grid_RF.best_score_) 
print ("BEST PARAMETERS:",grid_RF.best_params_)


#best_model = grid_DecisionTree.best_estimator_
#y_pred = best_model.predict(X_test)
#The accuracy of the test set:
#print ("The test set accuracy is:",accuracy_score(y_test, y_pred))

Best score is: 0.8791104188300449
BEST PARAMETERS: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 90, 'max_features': 2, 'n_estimators': 200}


In [25]:
Tuned_RF_model = RandomForestClassifier(**grid_RF.best_params_).fit(X_train,y_train)
Tuned_RF_y_prediction= Tuned_RF_model.predict(X_test)

HP_accuracy1_RF = Tuned_RF_model.score(X_train, y_train)
print("Accuracy of train data = ", HP_accuracy1_RF * 100, "%")
HP_accuracy2_RF = Tuned_RF_model.score(X_test, y_test)
print("Accuracy of test data = ", HP_accuracy2_RF * 100, "%")

#print ("Accuracy : ", accuracy_score(y_test, Tuned_RF_y_prediction)*100,"%")

Accuracy of train data =  100.0 %
Accuracy of test data =  87.01298701298701 %


### **KNN**

In [26]:
KNN_model=KNeighborsClassifier()
KNN_model.fit(X_train , y_train)
y_pred_KNN=KNN_model.predict(X_test)

accuracy1_KNN = KNN_model.score(X_train, y_train)
print("Accuracy of train data = ", accuracy1_KNN * 100, "%")
accuracy2_KNN = KNN_model.score(X_test, y_test)
print("Accuracy of test data = ", accuracy2_KNN * 100, "%")

Accuracy of train data =  88.64059590316573 %
Accuracy of test data =  85.28138528138528 %


In [27]:
k_range = list(range(1, 31))
param_grid_KNN = dict(n_neighbors=k_range)

grid_KNN = GridSearchCV(KNeighborsClassifier(), param_grid_KNN, cv=5, n_jobs=-1,scoring = 'accuracy' ).fit(X_train,y_train)


print ("Best score is:",grid_KNN.best_score_) 
print ("BEST PARAMETERS:",grid_KNN.best_params_)


Best score is: 0.8697473174108689
BEST PARAMETERS: {'n_neighbors': 2}


In [28]:
Tuned_KNN_model = KNeighborsClassifier(**grid_KNN.best_params_).fit(X_train,y_train)
Tuned_KNN_y_prediction= Tuned_KNN_model.predict(X_test)

HP_accuracy1_KNN = Tuned_KNN_model.score(X_train, y_train)
print("Accuracy of train data = ", HP_accuracy1_KNN * 100, "%")
HP_accuracy2_KNN = Tuned_KNN_model.score(X_test, y_test)
print("Accuracy of test data = ", HP_accuracy2_KNN * 100, "%")

#print ("Accuracy : ", accuracy_score(y_test, Tuned_KNN_y_prediction)*100,"%")

Accuracy of train data =  92.17877094972067 %
Accuracy of test data =  86.14718614718615 %
